In [1]:
# Импорт необходимых библиотек и модулей
from scipy.fftpack import fft, ifft, fftshift
import numpy as np
import matplotlib.pyplot as plt
import IPython
from scipy.io import wavfile

# Задание частот клавиш телефонной клавиатуры
rowTones = np.array([697, 770, 852, 941])
colTones = np.array([1209, 1336, 1477])
# Определение матрицы цифр на телефоне
numbers = [['1', '2', '3'],
           ['4', '5', '6'],
           ['7', '8', '9'],
           ['*', '0', '#']]

# Функция для нахождения ближайшего значения в массиве к заданному
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

# Функция для декодирования сигнала
def decode(sig):
    w = fft(sig) # Выполняется быстрое преобразование Фурье (БПФ) для получения спектра сигнала
    A = np.abs(w) # Модуль спектра
    freqs = np.fft.fftfreq(len(w)) # Вычисление частот
    ind = np.argpartition(A, -4)[-4:] # Индексы 4 наибольших значений в спектре
    freq = freqs[ind] # Соответствующие частоты
    freq.sort() # Сортировка частот
    freq = freq[2:] # Оставляем только 2 наиболее значимые частоты

    real_freq = freq * fs # Переводим относительные частоты в реальные

    # Определяем ближайшие ряд и столбец по частотам
    row = find_nearest(rowTones, real_freq[0])
    col = find_nearest(colTones, real_freq[1])

    mistake = 10 # Погрешность

    # Если расстояние между частотой и частотой ряда больше погрешности, возвращаем точку
    if np.abs(rowTones[row] - real_freq[0]) > mistake:
        return '.'
    # Если расстояние между частотой и частотой столбца больше погрешности, возвращаем точку
    if np.abs(colTones[col] - real_freq[1]) > mistake:
        return '.'

    # Возвращаем соответствующую цифру на телефонной клавиатуре
    return numbers[row][col]

# Параметры для анализа сигнала
sampleN = 500
xLim = 2000
S = []

# Цикл для обработки файлов с записями телефонных номеров
for i in range(10):
    fs, sig = wavfile.read('/Users/sasha/Desktop/signals/phoneNumber'+str(i)+'.wav') # Чтение файла с аудиосигналом
    sig = np.array(sig) # Преобразование сигнала в массив numpy
    N = sig.shape[0] # Длина сигнала
    duration = N / fs # Продолжительность сигнала в секундах
    amplitude = np.abs(fftshift(fft(sig))) # Амплитудный спектр сигнала
    s = '' # Переменная для хранения распознанных цифр
    # Цикл для обработки сегментов сигнала
    for k in range(int(N / sampleN)):
        sample = sig[k*sampleN : (k+1)*sampleN] # Выбираем сегмент сигнала для анализа
        sampleA = np.abs(fftshift(fft(sample))) # Вычисляем амплитудный спектр выбранного сегмента
        ratio = np.mean(sampleA) / np.median(sampleA) # Отношение средней амплитуды к медианной
        # Если отношение больше 1.2, сегмент считается содержащим сигнал и происходит его декодирование
        if ratio > 1.2:
            symbol = decode(sample)
            s += symbol
            print(symbol, end='') # Вывод распознанных символов на экран
        else:
            s += '.' # Если отношение меньше 1.2, сегмент считается шумом
            print(".", end='') # Вывод точки (обозначение шума) на экран
    S.append(s) # Добавление распознанного номера в список

# Печать распознанных номеров с удалением лишних точек
for s in S:
    for i in range(len(s)):
        if i == 0:
            print(s[i], end='') # Вывод первого символа без проверки на предшествующие точки
        if s[i-1] == '.' and s[i] != '.':
            print(s[i], end='') # Вывод символа, если предыдущий символ - точка, а текущий - не точка
    print() # Переход на новую строку после вывода каждого номера


888888..99999.22222..1111..99999..888888.9999..777777..999999...77777..777777..88888..999999..22222...1111..55555..33333..7777.1111..22222...44444...888888..8888..99999.....22..1111111..333333..999999..0000.5555...888888..3333..6666.888888..99999..000...5555.555555..55555.444444...44444..7777.888888.0000000..888888..9999...0000.5555555..2222...777...666666..444444..11111...888888.00000...8888..999999...00000..55555.111111...2222...555555..999999..999999..4444...2222...888888..99999..1111...1111..5555...000000..7777777..9999..3333...11111..555555.8888..99999...11111..1111.2222...88888..9999.333333..2222..2222..7777..8888..999999..11111..1111111..888..33333...11111.9999...00000..333..66666..88888..99999..111...111..7777777..4444...55555...00000...111111..66666..88888.889219897977
889215371248
889213905836
889055544780
889052764180
889051259942
889115079315
889112893227
889118319036
889117450168
